# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902044


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:37,  3.47s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:07,  2.49s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:46,  1.78s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:31,  1.28s/it]

...

Rendering models:  70%|███████   | 21/30 [00:08<00:01,  6.41it/s]

Rendering models:  80%|████████  | 24/30 [00:08<00:00,  8.13it/s]

Rendering models:  90%|█████████ | 27/30 [00:09<00:00,  9.67it/s]

Rendering models:  97%|█████████▋| 29/30 [00:09<00:00, 10.19it/s]

klmasters                             0.000318
not-logged-in-8c0c44faf754d344b18a    0.000222
mooberrysauce                         0.001544
ZigZagoon999                          0.000524
StephAChamber                         0.000252
equidad1                              0.127580
not-logged-in-8c0c44faf754d344b18a    0.000210
Thedunkmasta                          0.003371
ElisabethB                            0.112062
Alexander00                           0.000379
not-logged-in-9cdd68bb509752635e0b    0.002492
qbright                               0.000650
jmartinez468                          0.006792
TheEpicPrimius123                     0.000712
not-logged-in-478ba7e07e1b8664525f    0.004907
aidenr                                0.003886
Lavadude                              0.002555
Tessaje04                             2.787363
gmann2023                             0.002786
not-logged-in-e5015a872e4a0f43792d    0.000546
brigi.baumann                         0.000724
rmacmi2      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())